In [1]:
import sys
from os.path import join
gesladir = "/Users/alisonpeard/Documents/Oxford/DPhil/hybridmodels/GeslaDataset"
sys.path.append(gesladir)

# ESL data from Menéndez (2020)

In [4]:
import numpy as np
import geopandas as gpd
import folium
from itertools import product

esldir = join("4_FLOOD HEIGHT", "Regular Climate")
esls = gpd.read_file(join(esldir, "REGION_1_E1.shp"))
bbox = esls.total_bounds
xcoords = [bbox[0], bbox[2]]
ycoords = [bbox[1], bbox[3]]
bbox = [i for i in product(ycoords, xcoords)]
bbox.append(bbox[0])
center = [np.mean(ycoords), np.mean(xcoords)]

m=folium.Map(location=center, zoom_start=5, tiles='OpenStreetMap')
folium.Rectangle(bbox).add_to(m, name="Region")
m

If using pyshp instead of geopandas (hopefully not):

    import shapefile
    import folium
    from itertools import product

    esldir = join("4_FLOOD HEIGHT", "Regular Climate")
    roi = shapefile.Reader(join(esldir, "REGION_1_E1.shp"))
    point0 = roi.shape(0).__geo_interface__['coordinates'][::-1]  # convert to GeoJSON
    m=folium.Map(location=point0, zoom_start=5, tiles='OpenStreetMap')  # [28.644800, 77.216721])

    xcoords = [roi.bbox[0], roi.bbox[2]]
    ycoords = [roi.bbox[1], roi.bbox[3]]
    bbox = [i for i in product(ycoords, xcoords)]
    bbox.append(bbox[0])
    roi.bbox
    folium.Rectangle(bbox).add_to(m, name="Region")

    # for i, sf in enumerate(roi.iterShapes()):
    #     pass
    m

# GESLA dataset

In [5]:
from gesla import GeslaDataset

meta_file = join(gesladir, "GESLA3_ALL.csv")
data_path = join(gesladir, "GESLA3/")

g3 = GeslaDataset(meta_file=meta_file, data_path=data_path)

south_lat = 3  # 3.157889
north_lat = 20  # 19.56799
west_lon = 117  # 116.9271
east_lon = 127  # 127.1762

data = g3.load_lat_lon_range(
    south_lat=int(min(ycoords)),
    north_lat=int(max(ycoords)),
    west_lon=int(min(xcoords)),
    east_lon=int(max(xcoords)),
)

# make sure looks like right spot
print(data.site_name.values)

['Legaspi' 'Manila' 'Kota_Kinabalu' 'Tawau' 'Sandakan' 'Davao'
 'Puerto_Princesa' 'Cebu' 'Davao' 'Subic_Bay' 'Jolo' 'Currimao' 'Lubang']


In [8]:
from folium import plugins

df = data.to_dataframe()
df = df[["sea_level", "latitude", "longitude"]]
df = df.groupby("station").agg(np.mean)  # get average sea-levels
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

m = folium.Map(location = [15,30], tiles='Cartodb dark_matter', zoom_start = 2)
sea_levels = [[point.latitude, point.longitude, point.sea_level] for _, point in gdf.iterrows()]
plugins.HeatMap(sea_levels).add_to(m)

m

In [9]:
gdf.geometry

station
0     POINT (123.75800 13.14500)
1     POINT (120.96800 14.58500)
2      POINT (116.06700 5.98300)
3      POINT (117.88300 4.23300)
4      POINT (118.06700 5.81000)
5      POINT (125.66300 7.15300)
6      POINT (118.73300 9.75000)
7     POINT (123.91700 10.30000)
8      POINT (125.63300 7.08300)
9     POINT (120.25200 14.76500)
10     POINT (121.00000 6.06700)
11    POINT (120.48800 17.98800)
12    POINT (120.20200 13.81800)
Name: geometry, dtype: geometry

In [10]:
esls.geometry

0        POINT (123.99917 12.96792)
1        POINT (123.99914 12.96458)
2        POINT (123.99289 12.96747)
3        POINT (123.98581 12.96378)
4        POINT (123.97917 12.96208)
                    ...            
77195    POINT (120.02542 11.94331)
77196    POINT (120.02542 11.94417)
77197    POINT (120.02500 11.94456)
77198    POINT (120.02333 11.94458)
77199    POINT (120.02208 11.94583)
Name: geometry, Length: 77200, dtype: geometry

In [ ]:
def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
     nearest = gpd2.geometry == nearest_points(point, pts)[1]
     return gpd2[nearest].Place.get_values()[0]
gpd1['Nearest'] = gpd1.apply(lambda row: near(row.geometry), axis=1)
gpd1
    Name  ID     geometry  Nearest
0   John   1  POINT (1 1)     Home
1  Smith   1  POINT (2 2)    Shops
2   Soap   1  POINT (0 2)     Work

In [ ]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 
pts3 = esls.geometry.unary_union
pts3